In [1]:
import os
from modelscope.msdatasets import MsDataset
from modelscope.metainfo import Trainers, Preprocessors
from modelscope.utils.constant import ModelFile, Tasks
from modelscope.trainers import build_trainer
train_dataset = MsDataset.load('GeoGLUE', subset_name='GeoETA', split='train', namespace='damo')

KeyboardInterrupt: 

In [ ]:

train_dataset._hf_ds['validation']#.to_json('./data/train.json')

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 8856
})

In [ ]:
tmp_dir = 'tmp_dir'

def finetune(model_id,
             train_dataset,
             eval_dataset,
             name=Trainers.nlp_text_ranking_trainer,
             cfg_modify_fn=None,
             **kwargs):
    kwargs = dict(
        model=model_id,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        work_dir=tmp_dir,
        cfg_modify_fn=cfg_modify_fn,
        num_workers = 0,
        **kwargs)

    os.environ['LOCAL_RANK'] = '0'
    trainer = build_trainer(name=name, default_args=kwargs)
    trainer.train()
    results_files = os.listdir(tmp_dir)

In [ ]:
def cfg_modify_fn(cfg):
    cfg.task = 'token-classification'
    cfg['dataset'] = {
        'train': {
            'labels': label_enumerate_values,
            'first_sequence': 'tokens',
            'label': 'ner_tags',
            'sequence_length': 128
        }
    }
    cfg['preprocessor'] = {
        'type': 'token-cls-tokenizer',
        'padding': 'max_length'
    }
    cfg.train.max_epochs = 1
    cfg.train.dataloader.batch_size_per_gpu = 32
    cfg.train.optimizer.lr = 3e-5
    cfg.train.hooks = [
    {
        'type': 'CheckpointHook',
        'interval': 1
    },
    {
        'type': 'TextLoggerHook',
        'interval': 100
    }, {
        'type': 'IterTimerHook'
    }, {
        'type': 'EvaluationHook',
        'by_epoch': True
    }]
    cfg.train.lr_scheduler.total_iters = int(len(train_dataset) / 32) * cfg.train.max_epochs

    return cfg

In [ ]:
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list


In [ ]:
# load dataset
train_dataset = MsDataset.load('GeoGLUE', subset_name='GeoETA', split='train', namespace='damo')
dev_dataset = MsDataset.load('GeoGLUE', subset_name='GeoETA', split='validation', namespace='damo')

2023-04-22 18:20:44,994 - modelscope - INFO - Generating dataset GeoGLUE (C:\Users\pc\.cache\modelscope\hub\datasets\modelscope___GeoGLUE\master\train)
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 499.86it/s]
Using custom data configuration default-5533a47dc7c53f5a
Found cached dataset json (C:/Users/pc/.cache/huggingface/datasets/json/default-5533a47dc7c53f5a/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 83.34it/s]
2023-04-22 18:22:06,595 - modelscope - INFO - Generating dataset GeoGLUE (C:\Users\pc\.cache\modelscope\hub\datasets\modelscope___GeoGLUE\master\validation)
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 499.98it/s]
Using custom data configuration default-ca46828c1da0d3c1
Found cached dataset json (C:/Users/pc/.cache/huggingface/datasets/json/default-ca46828c1da0d3c1/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 90.91it/s]


In [ ]:
dev_dataset._hf_ds['validation'].to_json('./data/val.json')

# label_enumerate_values = get_label_list(train_dataset._hf_ds['train']['ner_tags'] + dev_dataset._hf_ds['validation']['ner_tags'])
# label_enumerate_values

Creating json from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 117.68ba/s]


661292

In [ ]:
model_id = 'damo/mgeo_backbone_chinese_base'
finetune(
    model_id=model_id,
    train_dataset=train_dataset['train'],
    eval_dataset=dev_dataset['validation'],
    cfg_modify_fn=cfg_modify_fn,
    name='nlp-base-trainer')

2023-04-21 22:55:33,897 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.1
2023-04-21 22:55:34,754 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.1
2023-04-21 22:55:35,284 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.1
2023-04-21 22:55:35,597 - modelscope - INFO - initialize model from C:\Users\pc\.cache\modelscope\hub\damo\mgeo_backbone_chinese_base
2023-04-21 22:55:38,203 - modelscope - WARNING - ('CUSTOM_DATASETS', 'token-classification', 'mgeo') not found in ast index file
2023-04-21 22:55:38,206 - modelscope - WARNING - ('CUSTOM_DATASETS', 'token-classification', 'mgeo') not found in ast index file
2023-04-21 22:55:38,207 - modelscope - INFO - cuda is not available, using cpu instead.
2023-04-21 22:55:38,209 - modelscope - INFO - ==========================Training Config Start==========================
2023-04-21 22:55:38,210 - modelscope - INFO - {
    "framework": "pytorch",


** build_dataset error log: 'mgeo is not in the custom_datasets registry group token-classification. Please make sure the correct version of ModelScope library is used.'
** build_dataset error log: 'mgeo is not in the custom_datasets registry group token-classification. Please make sure the correct version of ModelScope library is used.'


BrokenPipeError: [Errno 32] Broken pipe